In [ ]:
from IPython.external import mathjax;
mathjax.install_mathjax()

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

Extracting to /Users/toz/.ipython/nbextensions/mathjax

In [4]:
cd ../

C:\Users\Talha\Documents\WinPython3\projects\cnnyt


In [56]:
# Read the datasets
outlets = "ABC NBCNews CBSNews FoxNews AP WPOST".split()
dfs = [pd.read_csv('data/'+outlet+'-news.csv') for outlet in outlets]

In [57]:
#unify the datasets
df = pd.DataFrame(columns=dfs[0].columns)
for i,df2 in enumerate(dfs):
    df2['outlet'] = outlets[i]
    df = df.append(df2)

In [58]:
#calculate the sentiments of the tweeted news
dfs[0].columns

Index(['fname', 'dt', 'title', 'href', 'newstxt', 'outlet'], dtype='object')

In [59]:
df.shape

(12586, 6)

In [61]:
#sns.distplot(d, hist=False, color="g", kde_kws={"shade": True})
df.head()

,dt,fname,href,newstxt,outlet,title
0,2015-01-28 00:00:00,15Io0P8,http://abcnews.go.com/Technology/nasa-photos-s...,As the Northeast digs out from a wild winter s...,ABC,Blizzard 2015: NASA Photos Show Scope of Winte...
1,2015-01-27 00:00:00,15JlKXr,http://abcnews.go.com/International/russia-evi...,Russia has rejected accusations that one of it...,ABC,"Russia Says 'No Evidence' of NYC Spying, Despi..."
2,2015-01-29 00:00:00,15JQAzy,http://abcnews.go.com/Health/common-counter-me...,Common over-the-counter drugs such as Benadryl...,ABC,Common Over-the-Counter Medicines Linked to De...
3,2015-02-01 00:00:00,15NFjxR,http://abcnews.go.com/Sports/super-bowl-xlix-f...,Super Bowl XLIX between the New England Patrio...,ABC,Super Bowl 2015: Facts and Figures Ahead of th...
4,2015-01-27 00:00:00,15P0Qaz,http://abcnews.go.com/International/isis-hosta...,In a chilling new video from the terror group ...,ABC,ISIS Hostage Says He'll Be Killed Within 24 Ho...


In [78]:
import textblob as tb
df = df.fillna('')
blob = df.newstxt.apply(tb.TextBlob)
df['sp'] = blob.apply(lambda x: x.sentiment.polarity)

In [80]:
tws = [pd.read_csv('data/tweets/'+outlet+'.csv') for outlet in outlets]

In [84]:
tw = pd.DataFrame(columns=tws[0].columns)
for i,df2 in enumerate(tws):
    df2['outlet'] = outlets[i]
    tw = tw.append(df2)

In [95]:
tw.shape

(19199, 7)

In [127]:
tw = tw.fillna('')
tw['url']=tw['url'].str.rstrip('/')
tw['fname']=tw.apply(lambda x: x.url.split('/')[-1],axis=1)

In [143]:
m1 = tw.merge(df,on=['fname','outlet'],how='inner')
m1 = m1.drop_duplicates(subset=['fname','outlet'])

In [146]:
m1.to_csv('data/outlets6.csv',index=False)

In [164]:
cnnyt = pd.read_csv('data/all-sentipol.csv')
cnnyt = cnnyt.rename(columns={'spnews':'sp','src':'outlet'})
cnnyt = cnnyt.dropna(subset=['newstxt'])

In [165]:
cnnyt.columns

Index(['cat', 'dt', 'fav', 'href', 'newstxt', 'sp', 'rt', 'outlet', 'title', 'tweep', 'twtext', 'url', 'sptw', 'sptitle'], dtype='object')

In [167]:
#keep dt,href,outlet,rt,fav,sp
cols = ['dt','outlet','rt','fav','sp','href']
m2 = m1[cols].append(cnnyt[cols])

In [170]:
m2.to_csv('data/tw-sp.csv',index=False)